<a href="https://colab.research.google.com/github/NehaSontakk/BATH-Prokka-Comparison/blob/main/Updated_Contig_comparison_identifying_gaps_Prokka_vs_BATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alignment and Classification of BATH and Prokka Annotations

Performs a comprehensive comparison between genomic annotations produced by Prokka and BATH. First the BED files from Prokka and BATH annotation outputs are prepared, identifying overlaps and unique regions, and visualizing the results.

The code generates BED files for different genomic regions, segregates them based on DNA strands, and runs operations to find overlaps and unique annotations between the two datasets. It combines the resulting files by prefix, ensuring that all directories are created if they don't exist. Finally, the code analyzes the overlapping and unique regions, categorizes them, and visualizes the data with a Venn diagram to provide insights into the annotations.

In [ ]:
#Installations
!pip install gffpandas
!sudo apt-get install bedops
!pip install biopython

In [ ]:
#File paths
prokka_annotations = '/content/drive/MyDrive/Lab Work/Parkinsons_Data/BIN152/Prokka Output/bin152.gff'
bath_dedup_annotations = "/content/drive/MyDrive/Lab Work/Parkinsons_Data/BIN152/Deduplication Output/dedup_bin52_test1.xlsx"

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import subprocess
import matplotlib.pyplot as plt
import gffpandas.gffpandas as gffpd
from Bio import SeqIO
from glob import glob
from pathlib import Path

In [ ]:
#Load and prep the prokka files
annotation = gffpd.read_gff3(prokka_annotations)
combined_df = annotation.filter_feature_of_type(['CDS'])
attr_to_columns = combined_df.attributes_to_columns()
prokka_proteins = pd.DataFrame(attr_to_columns)
prokka_proteins['start'] = prokka_proteins['start'].astype(int)
prokka_proteins['end'] = prokka_proteins['end'].astype(int)

In [ ]:
prokka_proteins.head()

In [ ]:
#Load and prep bath files
bath_protein = pd.read_excel(bath_dedup_annotations)
unnamed_cols = [col for col in bath_protein.columns if col.startswith('Unnamed')]
bath_protein.drop(columns=unnamed_cols, inplace=True)
bath_protein['start'] = bath_protein['ali from']
bath_protein['end'] = bath_protein['ali to']
bath_protein['start'].fillna(bath_protein['ali from flip'],inplace=True)
bath_protein['end'].fillna(bath_protein['ali to flip'],inplace=True)

In [ ]:
bath_protein.head()

## Find the gaps filled by BATH vs Prokka per contig

In [ ]:
# Add 'ID' to bath_protein
bath_protein['ID'] = bath_protein['query name']

# Separate BATH data into positive and negative strands
pos_strand = bath_protein[bath_protein['strand'] == "+"].copy()
neg_strand = bath_protein[bath_protein['strand'] == "-"].copy()

# Handle columns for positive strand
pos_strand.loc[:, 'start'] = pos_strand['ali from'].astype(int)
pos_strand.loc[:, 'end'] = pos_strand['ali to'].astype(int)

# Handle columns for negative strand
neg_strand.loc[:, 'start'] = neg_strand['ali from flip'].astype(int)
neg_strand.loc[:, 'end'] = neg_strand['ali to flip'].astype(int)

# Filter Prokka proteins into positive and negative strands
prokka_proteins_plus = prokka_proteins[prokka_proteins['strand'] == '+'].copy()
prokka_proteins_minus = prokka_proteins[prokka_proteins['strand'] == '-'].copy()

# Sort and save positive strand Prokka proteins
prokka_proteins_plus.sort_values('start', inplace=True)
prokka_proteins_plus.to_csv('prokka_proteins_plus.bed', sep='\t', index=False, header=False, columns=['seq_id', 'start', 'end', 'strand', 'ID', 'inference'])

# Sort and save negative strand Prokka proteins
prokka_proteins_minus.sort_values('start', inplace=True)
prokka_proteins_minus.to_csv('prokka_proteins_minus.bed', sep='\t', index=False, header=False, columns=['seq_id', 'start', 'end', 'strand', 'ID', 'inference'])

# Sort and save BATH deduplicated positive strand
bath_deduplicated_plus = pos_strand[['target name', 'start', 'end', 'strand', 'ID', 'shifts']].copy()
bath_deduplicated_plus.sort_values('start', inplace=True)
bath_deduplicated_plus.to_csv('bath_deduplicated_plus.bed', sep='\t', index=False, header=False)

# Sort and save BATH deduplicated negative strand
bath_deduplicated_minus = neg_strand[['target name', 'start', 'end', 'strand', 'ID', 'shifts']].copy()
bath_deduplicated_minus.sort_values('start', inplace=True)
bath_deduplicated_minus.to_csv('bath_deduplicated_minus.bed', sep='\t', index=False, header=False)


### Functions



1.   generate_bed_files:
This function creates BED files by iterating through unique contig identifiers in the DataFrame, filtering and sorting data by start positions, and writing the data to BED files.

2.   run_bedmap_operations:
This function uses bedmap and bedops tools to find overlapping and unique regions between BATH and Prokka BED files. It generates three output files: one for overlaps, one for regions unique to BATH, and one for regions unique to Prokka, saving them in the specified output directory

3.   find_and_process_bed_pairs:
This function finds and pairs BED files from BATH and Prokka directories based on sequence identifiers. It then calls run_bedmap_operations to process each pair, identifying overlaps and unique regions, and saves the results in the output directory.

4.   combine_files_by_prefix:
This function organizes and combines BED files in a source directory by their prefixes. It groups files with the same prefix, concatenates their contents, and writes the combined data into new BED files in the output directory. Each new file is named with the prefix followed by "combined.bed".

In [ ]:
#Functions

#generate BED files from a dataframe for each unique sequence identifier (contig) present in the dataframe.
def generate_bed_files(df, directory, contig_column, file_prefix='Prokka_annotation'):
    # Generates BED files for each unique 'seq_id' in the DataFrame.
    os.makedirs(directory, exist_ok=True)
    for seq_id in df[contig_column].unique():
        df_seq = df[df[contig_column] == seq_id]
        #print(df_seq.shape)
        df_seq = df[df[contig_column] == seq_id].sort_values(by=['start'])
        filename = f'{directory}/{file_prefix}_{seq_id}.bed'
        with open(filename, 'w') as file:
            for _, row in df_seq.iterrows():
              #For a prokka file
              if file_prefix == "Prokka_annotation":
                bed_line = f"{row[contig_column]}\t{row['start']}\t{row['end']}\t{row['ID']}\t{row['strand']}\t{row['inference']}\n"
                file.write(bed_line)
              else:
                #For a BATH file
                bed_line = f"{row[contig_column]}\t{row['start']}\t{row['end']}\t{row['ID']}\t{row['strand']}\t{row['shifts']}\n"
                file.write(bed_line)

    print(f"BED files have been successfully generated in {directory}")

def run_bedmap_operations(bath_bed_path, prokka_bed_path, output_base_dir):
    seq_id = bath_bed_path.stem.split('_')[-1]
    overlap_output = output_base_dir / f'overlap_prokka_bath_{seq_id}.bed'
    unique_to_bath_output = output_base_dir / f'unique_to_bath_{seq_id}.bed'
    unique_to_prokka_output = output_base_dir / f'unique_to_prokka_{seq_id}.bed'

    output_base_dir.mkdir(parents=True, exist_ok=True)

    # Command for bedmap to find overlaps
    command = [
        "bedmap", "--echo", "--echo-map", "--delim", ";",
        "--fraction-ref", "0.15",
        str(bath_bed_path), str(prokka_bed_path)
    ]

    # Run the command and capture output
    result = subprocess.run(command, capture_output=True, text=True, check=True)

    # Filter the output and write to overlap_output
    with open(overlap_output, 'w') as f_output:
        for line in result.stdout.splitlines():
            parts = line.split(';')
            # Check if there is more than one part and the second part is not empty
            if len(parts) > 1 and parts[1].strip():
                f_output.write(line + '\n')

    # Run bedmap for unique to BATH
    subprocess.run(["bedops", "--not-element-of", "1", str(bath_bed_path), str(prokka_bed_path)], stdout=open(unique_to_bath_output, 'w'), check=True)

    # Run bedmap for unique to Prokka
    subprocess.run(["bedops", "--not-element-of", "1", str(prokka_bed_path), str(bath_bed_path)], stdout=open(unique_to_prokka_output, 'w'), check=True)

def find_and_process_bed_pairs(bath_dir, prokka_dir, output_dir):
    bath_dir_path = Path(bath_dir)
    prokka_dir_path = Path(prokka_dir)
    output_base_dir = Path(output_dir)

    bath_bed_files = {f.stem.split('_')[-1]: f for f in bath_dir_path.glob('BATH_annotation_*.bed')}
    prokka_bed_files = {f.stem.split('_')[-1]: f for f in prokka_dir_path.glob('Prokka_annotation_*.bed')}

    for seq_id, bath_bed_path in bath_bed_files.items():
        print(seq_id)
        prokka_bed_path = prokka_bed_files.get(seq_id)
        print("Processing files ... ",prokka_bed_path, "and ",bath_bed_path)
        if prokka_bed_path:
            run_bedmap_operations(bath_bed_path, prokka_bed_path, output_base_dir)
        else:
            print(f"No matching Prokka file found for BATH seq_id: {seq_id}")

def combine_files_by_prefix(source_directory, output_directory, prefixes):
    source_directory = Path(source_directory)
    output_directory = Path(output_directory)
    output_directory.mkdir(parents=True, exist_ok=True)
    file_groups = {prefix: [] for prefix in prefixes}
    for file_path in source_directory.glob('*.bed'):
        for prefix in prefixes:
            if file_path.name.startswith(prefix):
                file_groups[prefix].append(file_path)
    for prefix, files in file_groups.items():
        combined_file_path = output_directory / f"{prefix}combined.bed"
        with combined_file_path.open('w') as combined_file:
            for file_path in files:
                with file_path.open() as input_file:
                    combined_file.write(input_file.read())
                combined_file.write('\n')

In [ ]:
import os

BASE_DIR = '/content/BED'
PROKKA_PLUS_DIR = f'{BASE_DIR}/Prokka_Annotation_PLUS_BED'
PROKKA_MINUS_DIR = f'{BASE_DIR}/Prokka_Annotation_MINUS_BED'
BATH_PLUS_DIR = f'{BASE_DIR}/BATH_Annotation_PLUS_BED'
BATH_MINUS_DIR = f'{BASE_DIR}/BATH_Annotation_MINUS_BED'
OVERLAPS_DIR = f'{BASE_DIR}/Overlaps and Differences'
COMBINED_FILES_DIR = f'{BASE_DIR}/Combined Files'


def ensure_directories():
    directories = [
        PROKKA_PLUS_DIR, PROKKA_MINUS_DIR,
        BATH_PLUS_DIR, BATH_MINUS_DIR,
        OVERLAPS_DIR, COMBINED_FILES_DIR
    ]
    for directory in directories:
        os.makedirs(directory, exist_ok=True)



def generate_all_bed_files():
    generate_bed_files(prokka_proteins_plus, PROKKA_PLUS_DIR, 'seq_id', file_prefix='Prokka_annotation')
    generate_bed_files(prokka_proteins_minus, PROKKA_MINUS_DIR, 'seq_id', file_prefix='Prokka_annotation')
    generate_bed_files(bath_deduplicated_plus, BATH_PLUS_DIR, 'target name', file_prefix='BATH_annotation')
    generate_bed_files(bath_deduplicated_minus, BATH_MINUS_DIR, 'target name', file_prefix='BATH_annotation')

# Process BED pairs
def process_bed_pairs():
    find_and_process_bed_pairs(BATH_PLUS_DIR, PROKKA_PLUS_DIR, OVERLAPS_DIR)

def combine_bed_files():
    prefixes = ['unique_to_prokka_', 'unique_to_bath_', 'overlap_prokka_bath_']
    combine_files_by_prefix(OVERLAPS_DIR, COMBINED_FILES_DIR, prefixes)

ensure_directories()
generate_all_bed_files()
process_bed_pairs()
combine_bed_files()


### Analysis

#### Venn diagram of region based overlaps

In [ ]:
def transform_file_to_dataframe(file_path):
    transformed_rows = []

    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            fields = line.strip().split(';')
            if len(fields) == 3:
                new_row1 = [fields[0], fields[1]]
                new_row2 = [fields[0], fields[2]]
                transformed_rows.append(new_row1)
                transformed_rows.append(new_row2)
            else:
                transformed_rows.append(fields)
    df = pd.DataFrame(transformed_rows, columns=['BATH', 'Prokka']).replace("\n",np.nan).dropna()

    return df

In [ ]:
file_path = '/content/BED/Combined Files/overlap_prokka_bath_combined.bed'
overlaps = transform_file_to_dataframe(file_path)

In [ ]:
def determine_label(row):
    if row['Prokka'].endswith('ab initio prediction:Prodigal:002006'):
        return "BATH and Prokka Unannotated"
    else:
        return "BATH and Prokka Annotated"

overlaps['label'] = overlaps.apply(determine_label, axis=1)

In [ ]:
file_path = '/content/BED/Combined Files/unique_to_bath_combined.bed'
with open(file_path, 'r') as file:
    lines = [line.strip() for line in file.readlines() if line.strip()]
unique_to_bath = pd.DataFrame(lines, columns=['BATH']).replace("\n",np.nan).dropna()
unique_to_bath['Prokka'] = np.nan
unique_to_bath['label'] = 'BATH'

In [ ]:
file_path3 = '/content/BED/Combined Files/unique_to_prokka_combined.bed'
with open(file_path3, 'r') as file:
    lines = [line.strip() for line in file.readlines() if line.strip()]
unique_to_prokka = pd.DataFrame(lines, columns=['Prokka']).replace("\n",np.nan).dropna()
unique_to_prokka['BATH'] = np.nan

def determine_label2(row):
    if row['Prokka'].endswith('ab initio prediction:Prodigal:002006'):
        return "Prokka Unannotated"
    else:
        return "Prokka Annotated"

unique_to_prokka['label'] = unique_to_prokka.apply(determine_label2, axis=1)

In [ ]:
venn_data = pd.concat([unique_to_prokka,unique_to_bath,overlaps])

In [ ]:
venn_data.to_excel("/content/drive/MyDrive/Lab Work/Parkinsons_Data/BIN152/Aligned Outputs/Aligned_Outputs_bin152.xlsx",index=False)

In [ ]:
venn_data['label'].value_counts()

In [ ]:
venn_data[venn_data['label'] == "Prokka Annotated"]

In [ ]:
from matplotlib_venn import venn3
import matplotlib.pyplot as plt

# Set the default style for clearer and more vibrant visuals
plt.style.use('seaborn')
venn_counts = {
    '100': venn_data[venn_data['label'] == "BATH"].shape[0],  # Only BATH
    '010': venn_data[venn_data['label'] == "Prokka Annotated"].shape[0],  # Only Prokka Annotated
    '001': venn_data[venn_data['label'] == "Prokka Unannotated"].shape[0],  # Only Prokka Unannotated
    '110': venn_data[venn_data['label'] == "BATH and Prokka Annotated"].shape[0],  # Intersection BATH and Prokka Annotated
    '101': venn_data[venn_data['label'] == "BATH and Prokka Unannotated"].shape[0],  # Intersection BATH and Prokka Unannotated
    '011': 0,  # Intersection of Prokka Annotated and Prokka Unannotated (not given)
    '111': 0   # Intersection of all three (not applicable)
}

# Custom colors for the sets
set_colors = ("#de3028", "#1B263B", "#FFD700")

# Creating the Venn diagram with custom colors
venn3(subsets=venn_counts, set_labels=('BATH Annotations', 'Prokka Annotations', 'ORFs (Prodigal)'), set_colors=set_colors)


plt.title("Overlaping annotations BATH and Prokka based DNA position")
plt.show()


In [ ]:
#You need to find the areas annotated by BATH in regions that only prodigal annotates
#You also need to find BATH individual areas

In [ ]:
#You also need to quantify it somehow